<a href="https://colab.research.google.com/github/fecsaba/ComfyUI/blob/main/Flux1_dev_lora_init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Notes
- Uncheck `UPDATE_COMFY_UI`, and `USE_COMFYUI_MANAGER` and `INSTALL_GGUF_NODE` after first run if `USE_GOOGLE_DRIVE` is checked.
## Crystools
## EasyUse
## ITools
## Was node suite
## ComfyUI's ControlNet Auxiliary Preprocessors
## ComfyUI ControlAltAI Nodes

# INIT

In [ ]:
# #@title Environment Setup
from pathlib import Path
import os

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_GGUF_NODE = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}

OPTIONS.update({
    'USE_GOOGLE_DRIVE': USE_GOOGLE_DRIVE,
    'UPDATE_COMFY_UI': UPDATE_COMFY_UI,
    'USE_COMFYUI_MANAGER': USE_COMFYUI_MANAGER,
    'INSTALL_GGUF_NODE': INSTALL_GGUF_NODE,
    'INSTALL_CUSTOM_NODES_DEPENDENCIES': INSTALL_CUSTOM_NODES_DEPENDENCIES
})

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /
    from google.colab import drive
    drive.mount('/content/drive')
    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

!export COMFYUI_PATH=WORKSPACE

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !chmod 755 .ci/**/*.bat .ci/**/*.py .ci/**/*.txt
  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde kornia>=0.7.1 spandrel soundfile sentencepiece

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes
  !chmod 755 ComfyUI-Manager/**/*.sh
  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

if OPTIONS['INSTALL_GGUF_NODE']:
  %cd $WORKSPACE/custom_nodes
  !git clone https://github.com/city96/ComfyUI-GGUF
  !pip install -r ComfyUI-GGUF/requirements.txt

%cd $WORKSPACE

!echo -= Install custom nodes dependencies =-
!pip install GitPython
!python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

!echo -= Install nodes =-
!echo -= **************** =-
!echo -= Install Crystools =-
%cd $WORKSPACE/custom_nodes
!git clone https://github.com/crystian/ComfyUI-Crystools.git
!pip install -r ComfyUI-Crystools/requirements.txt

!echo -= Install EasyUse =-
%cd $WORKSPACE/custom_nodes
!git clone https://github.com/yolain/ComfyUI-Easy-Use.git
!pip install -r ComfyUI-Easy-Use/requirements.txt

!echo -= Install Itools =-

def install_custom_node(comfyui_path, repo_url, node_folder_name):
    custom_nodes_path = os.path.join(comfyui_path, "custom_nodes")

    if not os.path.exists(custom_nodes_path):
        os.makedirs(custom_nodes_path)
    print(f"custom_nodes mappa elérési útvonala: {custom_nodes_path}")

    destination_path = os.path.join(custom_nodes_path, node_folder_name)

    if not os.path.exists(destination_path):
        !git clone {repo_url} {destination_path}
        print(f"{node_folder_name} telepítése kész.")
    else:
        print(f"{node_folder_name} már telepítve van.")

    requirements_path = os.path.join(destination_path, "requirements.txt")
    if os.path.exists(requirements_path):
        !pip install -r {requirements_path}
        print("Dependenciák telepítve.")
    else:
        print("Nincs requirements.txt fájl, nem szükséges dependenciákat telepíteni.")

comfyui_path = "/content/drive/MyDrive/ComfyUI"
# iTools
repo_url = "https://github.com/fecsaba/ComfyUI-iTools.git"
node_folder_name = "ComfyUI-iTools"

install_custom_node(comfyui_path, repo_url, node_folder_name)

# Was node suite
repo_url = "https://github.com/WASasquatch/was-node-suite-comfyui.git"
node_folder_name = "was-node-suite-comfyui"

install_custom_node(comfyui_path, repo_url, node_folder_name)

# ComfyUI's ControlNet Auxiliary Preprocessors
repo_url = "https://github.com/Fannovel16/comfyui_controlnet_aux.git"
node_folder_name = "comfyui_controlnet_aux"

install_custom_node(comfyui_path, repo_url, node_folder_name)

# ComfyUI ControlAltAI Nodes
repo_url = "https://github.com/gseth/ControlAltAI-Nodes.git"
node_folder_name = "ControlAltAI-Nodes"

install_custom_node(comfyui_path, repo_url, node_folder_name)
print("Telepítés befejezve. Indítsd újra a ComfyUI-t!")